In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j  langchain-neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USERNAME = userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

NEO4J_URI = NEO4J_URI
NEO4J_USERNAME = NEO4J_USERNAME
NEO4J_PASSWORD = NEO4J_PASSWORD
GROQ_API_KEY = GROQ_API_KEY

In [3]:
import os


os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD


In [4]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=str(NEO4J_URI),
    username= str(NEO4J_USERNAME),
    password=str(NEO4J_PASSWORD),
)
graph

<ipython-input-4-6cc53d2d5161>:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(


In [6]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.6 MB/s eta 0:00:00


In [7]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)
llm_transformer

In [128]:
query = """
// Create Knowledge node
MERGE (knowledge:Knowledge {name: 'Knowledge'})

// Create and connect PYS
MERGE (pys:PYS {name: 'PYS'})
MERGE (knowledge)-[:HAS_SOURCE]->(pys)

// Create and connect GITA
MERGE (gita:GITA {name: 'GITA'})
MERGE (knowledge)-[:HAS_SOURCE]->(gita)
"""


In [129]:
graph.query(query)

[]

In [130]:
query = """
// =========================== PYS Dataset ===========================


// Load and process PYS dataset
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/parth1609/RAGveda/main/dataset/PYS_English_Questions.csv' AS row
WITH row
WHERE row.chapter IS NOT NULL AND row.chapter <> ''
  AND row.verse IS NOT NULL AND row.verse <> ''

// Ensure the Knowledge node exists
MERGE (knowledge:Knowledge {name: 'Knowledge'})

// Create the PYS label and connect it to Knowledge
MERGE (pys:PYS {name: 'PYS'})
MERGE (knowledge)-[:HAS_SOURCE]->(pys)

// Create PYS chapters and their relationships
MERGE (chapter:PYSChapter {name: row.chapter})
MERGE (pys)-[:HAS_CHAPTER]->(chapter)

// Create PYS verses and their relationships
MERGE (verse:PYSVerse {number: row.verse, chapterName: row.chapter})
MERGE (chapter)-[:HAS_VERSE]->(verse)

// Create PYS Sanskrit shlokas and their relationships
MERGE (shloka:PYSShloka {text: row.sanskrit, verseNumber: row.verse, chapterName: row.chapter})
MERGE (verse)-[:HAS_SHLOKA]->(shloka)

// Create PYS translations and their relationships
MERGE (translation:PYSTranslation {text: row.translation, verseNumber: row.verse, chapterName: row.chapter})
MERGE (shloka)-[:HAS_TRANSLATION]->(translation)

// Create PYS questions and their relationships
MERGE (question:PYSQuestion {text: row.question, translationText: row.translation, verseNumber: row.verse, chapterName: row.chapter})
MERGE (translation)-[:RELATED_QUESTION]->(question)
MERGE (question)-[:ABOUT]->(translation)
MERGE (question)-[:HAVING_VERSE]->(shloka)


"""

In [131]:
graph.query(query)

[]

In [96]:
graph

In [97]:
graph.query(query)

[]

In [146]:
query = """

// Load and process GITA dataset
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/parth1609/RAGveda/main/dataset/Gita_questions.csv' AS row
WITH row
WHERE row.chapter IS NOT NULL AND row.chapter <> ''
  AND row.verse IS NOT NULL AND row.verse <> ''

// Ensure the Knowledge node exists
MERGE (knowledge:Knowledge {name: 'Knowledge'})

// Create the GITA label and connect it to Knowledge
MERGE (gita:GITA {name: 'GITA'})
MERGE (knowledge)-[:HAS_SOURCE]->(gita)

// Create GITA chapters and their relationships
MERGE (chapter:GITAChapter {name: row.chapter})
MERGE (gita)-[:HAS_CHAPTER]->(chapter)

// Create GITA verses and their relationships
MERGE (verse:GITAVerse {number: row.verse, chapterName: row.chapter})
MERGE (chapter)-[:HAS_VERSE]->(verse)

// Create GITA Sanskrit shlokas and their relationships
MERGE (shloka:GITASanskritShloka {text: row.sanskrit, verseNumber: row.verse, chapterName: row.chapter})
MERGE (verse)-[:HAS_SHLOKA]->(shloka)

// Create GITA translations and their relationships
MERGE (translation:GITATranslation {text: row.translation, verseNumber: row.verse, chapterName: row.chapter})
MERGE (shloka)-[:HAS_TRANSLATION]->(translation)

// Create GITA questions and their relationships
MERGE (question:GITAQuestion {text: row.question, translationText: row.translation, verseNumber: row.verse, chapterName: row.chapter})
MERGE (translation)-[:RELATED_QUESTION]->(question)
MERGE (question)-[:ABOUT]->(translation)


// Connect GITA speakers to Sanskrit shlokas
MERGE (speaker:GITASpeaker {name: row.speaker})
MERGE (speaker)-[:SAID_SHLOKA]->(shloka)

"""

In [147]:
graph

In [148]:
graph.query(query)

[]

In [145]:
graph.refresh_schema()
print(graph.schema)

Node properties:
PYS {name: STRING}
Knowledge {name: STRING}
PYSChapter {name: STRING}
PYSVerse {number: STRING, chapterName: STRING}
PYSShloka {chapterName: STRING, verseNumber: STRING, text: STRING}
PYSTranslation {chapterName: STRING, verseNumber: STRING, text: STRING}
PYSQuestion {chapterName: STRING, verseNumber: STRING, text: STRING, translationText: STRING}
GITA {name: STRING}
GITAChapter {name: STRING}
GITAVerse {number: STRING, chapterName: STRING}
GITASanskritShloka {chapterName: STRING, verseNumber: STRING, text: STRING}
GITATranslation {chapterName: STRING, verseNumber: STRING, text: STRING}
GITAQuestion {chapterName: STRING, verseNumber: STRING, text: STRING, translationText: STRING}
GITASpeaker {name: STRING}
Relationship properties:

The relationships:
(:PYS)-[:HAS_CHAPTER]->(:PYSChapter)
(:Knowledge)-[:HAS_SOURCE]->(:PYS)
(:Knowledge)-[:HAS_SOURCE]->(:GITA)
(:PYSChapter)-[:HAS_VERSE]->(:PYSVerse)
(:PYSVerse)-[:HAS_SHLOKA]->(:PYSShloka)
(:PYSShloka)-[:HAS_TRANSLATION]->(

In [136]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True, allow_dangerous_requests=True) # Added allow_dangerous_requests=True

In [137]:
%pip install -qU langchain-groq

In [149]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    max_tokens=1000,
)

In [150]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True, allow_dangerous_requests=True)

chain


GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7ff24caf1240>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7ff211d65390>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7ff211fc8670>, temperature=1e-08, mode

In [162]:
question = "How does the Gita start?"
sanskritShloka = "धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः| मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय || 1.1 ||"
speaker = "अर्जुन"
response=chain.invoke({"query":f"what answer of this {question} ?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (gita:GITA)-[:HAS_CHAPTER]->(chapter:GITAChapter)-[:HAS_VERSE]->(verse:GITAVerse)-[:HAS_SHLOKA]->(shloka:GITASanskritShloka)
WHERE gita.name = 'GITA' AND chapter.name = 'Chapter 1' AND verse.number = '1'
RETURN shloka.text AS startShloka
Full Context:
[]

> Finished chain.


{'query': 'what answer of this How does the Gita start? ?',
 'result': "I don't have information on how the Gita starts."}